In [ ]:
from utils import *

hete = Heterogeneity()

In [ ]:
facies = hete.load_facies()

hete_fluv, hete_gaus = hete.load_perm_poro()

In [ ]:
fig, axs = plt.subplots(3, 10, figsize=(20,5))
im0, im1, im2 = {}, {}, {}
for j in range(10):
    k = j*75
    im0 = axs[0,j].imshow(facies[k], 'jet')
    im1 = axs[1,j].imshow(hete_fluv[k,:,:,0], 'jet')
    im2 = axs[2,j].imshow(hete_fluv[k,:,:,1], 'jet')
    for i in range(3):
        axs[i,j].set(xticks=[], yticks=[])
plt.colorbar(im0, fraction=0.046, pad=0.04)
plt.colorbar(im1, fraction=0.046, pad=0.04)
plt.colorbar(im2, fraction=0.046, pad=0.04)

plt.show()

***
# END